# Extracting value from the payment notes

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import pymongo
import json
import datetime
import pickle
import functions as fn
import matplotlib.pyplot as plt 
%matplotlib inline
import emoji
import regex
import nltk
from nltk import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords, wordnet
import string
from emoji.unicode_codes import UNICODE_EMOJI as ue
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora

## Info to access db

In [ ]:
# Retrieve information to connect to the database
keys = fn.get_keys("/Users/jjherranzsarrion/.secret/local_info.json")
username = keys['username']
password = keys['password']

# Select your time ranges and add them in variables
train_window_end = '2018-08-05 23:59:59'
test_window_start = '2018-08-06 00:00:00'
test_window_end = '2018-08-07 23:59:59'
previous_day_start = '2018-08-05 00:00:00'

## Select all payment notes in db for the train period

In [ ]:
user_notes = fn.extract_user_notes(username, password, train_window_end)

In [ ]:
notes = user_notes['note']

## Preprocess the text data

In [ ]:
recomposed_note, emoji_dict = fn.get_notes_into_unicode(notes)

In [ ]:
recomposed_note_stopped = fn.get_clean_text_pattern(recomposed_note)

In [ ]:
lemmatized_notes = fn.lemmatize_notes(recomposed_note_stopped)

In [ ]:
recomposed_note_stopped_em = fn.turn_emoji_unicode_to_text(lemmatized_notes, emoji_dict)

In [ ]:
fully_recomposed_notes = fn.emojis_to_text(recomposed_note_stopped_em)

## Vectorise the text using Doc2Vec method

In [ ]:
# Train the model (time consuming)
tagged_data = fn.train_doc2vec_vectorizer(fully_recomposed_notes)

In [ ]:
user_vectrs_df = fn.get_aggregated_user_note_vector(username, password, train_window_end)

In [ ]:
user_vectrs_df.head()

## Using k means clustering to determine the optimal number of clusters

In [ ]:
# Retrieve the distortion plot for the different number of clusters k
fn.get_distortion_plot(whole_corpus_notes)

__Optimal number of k's is 6__

## Using LDA model to derive the topic of each cluster

In [ ]:
# Fit the model and retrieve clusters (time consuming)
lda_model = fn.get_cluster_topics_with_LDA(recomposed_note_stopped_em)

The topics of the 6 clusters are:
- Food
- Money_with_wings emoji
- Pizza emoji
- Uber
- Clickling beer mugs emoji
- Beer mug

In [ ]:
# Infer topic distributions on new, unseen documents, with
doc_lda = lda_model[doc_bow]